In [1]:
import sys
sys.path.append("../")

import tensorflow as tf
import tensorflow_addons as tfa
from modules import utils, models
from tensorflow.keras import metrics
from tensorflow.keras import callbacks

2021-07-20 03:19:04.324055: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


# Training Single-Record Models:
## Metadata-Only Model:

In [2]:
metadata_train_gen = utils.CombinedDataGen(data_file='../data/master_train_df.csv',
                                           out_mode='meta',
                                           mode='train',
                                           shuffle=True,
                                           scaler_dir='../models/scalers')

metadata_valid_gen = utils.CombinedDataGen(data_file='../data/master_valid_df.csv',
                                           out_mode='meta',
                                           mode='valid',
                                           shuffle=True,
                                           scaler_dir='../models/scalers')

train_ds = utils.get_ds_from_gen(metadata_train_gen,
                                 out_mode='meta',
                                 mode='train',
                                 batch_size=64,
                                 cache_dir='')
valid_ds = utils.get_ds_from_gen(metadata_valid_gen,
                                 out_mode='meta',
                                 mode='valid',
                                 batch_size=64,
                                 cache_dir='')

2021-07-20 03:19:07.554780: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-07-20 03:19:07.645916: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-07-20 03:19:07.645958: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-07-20 03:19:07.667857: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-07-20 03:19:07.680802: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2021-07-20 03:19:07.691493: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

In [3]:
metadata_mdl = models.init_meta_model()

metadata_mdl.compile(optimizer='adam', 
                     loss='binary_crossentropy',
                     metrics=[metrics.AUC()])

In [4]:
es_cb = callbacks.EarlyStopping(monitor='val_loss',
                                min_delta=1e-8,
                                patience=10,
                                restore_best_weights=True)

metadata_mdl.fit(train_ds, 
                 epochs=10_000,
                 validation_data=valid_ds, 
                 callbacks=[es_cb])

Epoch 1/10000


2021-07-20 03:19:09.762859: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


194/194 [==============================] - 39s 203ms/step - loss: 0.6446 - auc: 0.6304 - val_loss: 0.6322 - val_auc: 0.6535
Epoch 2/10000
194/194 [==============================] - 4s 19ms/step - loss: 0.6261 - auc: 0.6710 - val_loss: 0.6208 - val_auc: 0.6722
Epoch 3/10000
194/194 [==============================] - 4s 19ms/step - loss: 0.6161 - auc: 0.6869 - val_loss: 0.6134 - val_auc: 0.6831
Epoch 4/10000
194/194 [==============================] - 4s 19ms/step - loss: 0.6098 - auc: 0.6961 - val_loss: 0.6089 - val_auc: 0.6885
Epoch 5/10000
194/194 [==============================] - 4s 19ms/step - loss: 0.6061 - auc: 0.7010 - val_loss: 0.6065 - val_auc: 0.6923
Epoch 6/10000
194/194 [==============================] - 4s 19ms/step - loss: 0.6037 - auc: 0.7040 - val_loss: 0.6050 - val_auc: 0.6943
Epoch 7/10000
194/194 [==============================] - 4s 19ms/step - loss: 0.6022 - auc: 0.7060 - val_loss: 0.6040 - val_auc: 0.6957
Epoch 8/10000
194/194 [==============================] - 4s 

## Image-Only Model:

In [6]:
import sys
sys.path.append("../")

import tensorflow as tf
import tensorflow_addons as tfa
from modules import utils, models
from tensorflow.keras import metrics
from tensorflow.keras import callbacks

In [7]:
image_train_gen = utils.CombinedDataGen(data_file='../data/master_train_df.csv',
                                        out_mode='image',
                                        mode='train',
                                        shuffle=True,
                                        scaler_dir='../models/scalers')

image_valid_gen = utils.CombinedDataGen(data_file='../data/master_valid_df.csv',
                                        out_mode='image',
                                        mode='valid',
                                        shuffle=True,
                                        scaler_dir='../models/scalers')

train_ds = utils.get_ds_from_gen(image_train_gen,
                                 out_mode='image',
                                 mode='train',
                                 batch_size=64,
                                 cache_dir='/home/yglaser/lus_scratch/tfdata_cache/im_train.tmp')
valid_ds = utils.get_ds_from_gen(image_valid_gen,
                                 out_mode='image',
                                 mode='valid',
                                 batch_size=64,
                                 cache_dir='/home/yglaser/lus_scratch/tfdata_cache/im_valid.tmp')

/home/yglaser/.conda/envs/bioenv/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py:493: UserWarning: fill_value is not supported and is always 0 for TensorFlow < 2.4.0.
  return py_builtins.overload_of(f)(*args)


In [8]:
image_mdl = models.init_image_model()

eps = 1_000
nb_steps = eps * (len(image_train_gen) // 64)

radam = tfa.optimizers.RectifiedAdam(lr=1e-3, 
                                     total_steps=nb_steps, 
                                     warmup_proportion=0.02, 
                                     min_lr=1e-5)
ranger = tfa.optimizers.Lookahead(radam, sync_period=7, slow_step_size=0.4)

image_mdl.compile(optimizer=ranger,
                  loss='binary_crossentropy',
                  metrics=[metrics.AUC()])

In [9]:
es_cb = callbacks.EarlyStopping(monitor='val_loss',
                                min_delta=1e-8,
                                patience=10,
                                restore_best_weights=True)

image_mdl.fit(train_ds, 
              epochs=eps,
              validation_data=valid_ds, 
              callbacks=[es_cb],
              verbose=1)

Epoch 1/1000


2021-07-20 03:26:15.527103: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2021-07-20 03:26:18.000459: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2021-07-20 03:26:18.120309: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


194/194 [==============================] - 1344s 7s/step - loss: 0.8661 - auc_1: 0.5001 - val_loss: 0.7878 - val_auc_1: 0.4845
Epoch 2/1000
194/194 [==============================] - 836s 4s/step - loss: 0.8250 - auc_1: 0.5053 - val_loss: 0.6833 - val_auc_1: 0.5176
Epoch 3/1000
194/194 [==============================] - 726s 4s/step - loss: 0.7958 - auc_1: 0.5109 - val_loss: 0.6768 - val_auc_1: 0.5179
Epoch 4/1000
194/194 [==============================] - 707s 4s/step - loss: 0.7892 - auc_1: 0.5004 - val_loss: 0.6695 - val_auc_1: 0.5322
Epoch 5/1000
194/194 [==============================] - 710s 4s/step - loss: 0.7684 - auc_1: 0.4995 - val_loss: 0.6782 - val_auc_1: 0.5643
Epoch 6/1000
194/194 [==============================] - 708s 4s/step - loss: 0.7404 - auc_1: 0.5031 - val_loss: 0.6911 - val_auc_1: 0.5392
Epoch 7/1000
194/194 [==============================] - 728s 4s/step - loss: 0.7236 - auc_1: 0.5076 - val_loss: 0.6757 - val_auc_1: 0.5766
Epoch 8/1000
194/194 [=================

In [10]:
image_mdl.save('../models/image_single_rec.h5')

## Combined Model:

In [1]:
import sys
sys.path.append("../")

import tensorflow as tf
import tensorflow_addons as tfa
from modules import utils, models
from tensorflow.keras import metrics
from tensorflow.keras import callbacks

2021-07-20 21:02:03.422983: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
combined_train_gen = utils.CombinedDataGen(data_file='../data/master_train_df.csv',
                                           out_mode='combined',
                                           mode='train',
                                           shuffle=True,
                                           scaler_dir='../models/scalers')

combined_valid_gen = utils.CombinedDataGen(data_file='../data/master_valid_df.csv',
                                           out_mode='combined',
                                           mode='valid',
                                           shuffle=True,
                                           scaler_dir='../models/scalers')

train_ds = utils.get_ds_from_gen(combined_train_gen,
                                 out_mode='combined',
                                 mode='train',
                                 batch_size=64,
                                 cache_dir='/home/yglaser/lus_scratch/tfdata_cache/comb_train.tmp')
valid_ds = utils.get_ds_from_gen(combined_train_gen,
                                 out_mode='combined',
                                 mode='valid',
                                 batch_size=64,
                                 cache_dir='/home/yglaser/lus_scratch/tfdata_cache/comb_valid.tmp')

2021-07-20 21:02:21.157482: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-07-20 21:02:21.282398: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-07-20 21:02:21.282438: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-07-20 21:02:21.423095: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-07-20 21:02:21.507013: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2021-07-20 21:02:21.770799: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

In [5]:
combined_mdl = models.init_combined_model(weights='../models/image_single_rec.h5')

for l in combined_mdl.layers:
    assert l.trainable
    if l.name.startswith('img'):
        l.trainable = False
    if l.name.startswith('meta'):
        l.trainable = False
    if l.name.startswith('densenet'):
        for ll in l.layers:
            ll.trainable = False

combined_mdl.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=[metrics.AUC()])

In [ ]:
es_cb = callbacks.EarlyStopping(monitor='val_loss',
                                min_delta=1e-8,
                                patience=10,
                                restore_best_weights=True)

combined_mdl.fit(train_ds,
                 epochs=30,
                 validation_data=valid_ds, 
                 callbacks=[es_cb],
                 verbose=1)

Epoch 1/30
194/194 [==============================] - 1437s 7s/step - loss: 0.6745 - auc_1: 0.5384 - val_loss: 0.6578 - val_auc_1: 0.6319
Epoch 2/30
194/194 [==============================] - 1219s 6s/step - loss: 0.6715 - auc_1: 0.5452 - val_loss: 0.6552 - val_auc_1: 0.6340
Epoch 3/30
194/194 [==============================] - 1229s 6s/step - loss: 0.6610 - auc_1: 0.5843 - val_loss: 0.6586 - val_auc_1: 0.6459
Epoch 4/30
194/194 [==============================] - 1233s 6s/step - loss: 0.6537 - auc_1: 0.6070 - val_loss: 0.6414 - val_auc_1: 0.6537
Epoch 5/30
194/194 [==============================] - 1237s 6s/step - loss: 0.6484 - auc_1: 0.6215 - val_loss: 0.6341 - val_auc_1: 0.6668
Epoch 6/30
 69/194 [=========>....................] - ETA: 8:43 - loss: 0.6489 - auc_1: 0.6161

In [ ]:
eps = 1_000
nb_steps = eps * (len(image_train_gen) // 64)

radam = tfa.optimizers.RectifiedAdam(lr=1e-3, 
                                     total_steps=nb_steps, 
                                     warmup_proportion=0.012, 
                                     min_lr=1e-5)
ranger = tfa.optimizers.Lookahead(radam, sync_period=7, slow_step_size=0.4)


for l in combined_mdl.layers:
    assert l.trainable
    if l.name.startswith('img'):
        l.trainable = False
    if l.name.startswith('meta'):
        l.trainable = False
    if l.name.startswith('densenet'):
        for ll in l.layers:
            ll.trainable = False
            
combined_mdl.compile(optimizer=ranger,
                     loss='binary_crossentropy',
                     metrics=[metrics.AUC()])

In [ ]:
combined_mdl.fit(train_ds,
                 epochs=eps,
                 validation_data=valid_ds, 
                 callbacks=[es_cb],
                 verbose=1)

In [ ]:
combined_mdl.save('../models/combined_single_rec.h5')